# Python Standard Library Project - Boston Advisory 

### By: Okonkwo Obinna Uzochukwu

### Project Python Standard Library (***Iteration***)

For this project you have 4 files containing information about persons.

The files are:

    personal_info.csv - personal information such as name, gender, etc. (one row per person)
    vehicles.csv - what vehicle people own (one row per person)
    employment.csv - where a person is employed (one row per person)
    update_status.csv - when the person's data was created and last updated

Each file contains a key, SSN, which uniquely identifies a person.

This key is present in all four files.

You are guaranteed that the same SSN value is present in every file, and that it only appears once per file.

In addition, the files are all sorted by SSN, i.e. the SSN values appear in the same order in each file.


##### Goal 1

Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

For now these four iterators are just separate, independent iterators.

##### Goal 2

Create a single iterable that combines all the columns from all the iterators.

The iterable should yield named tuples containing all the columns.
Make sure that the SSN's across the files match!

All the files are guaranteed to be in SSN sort order, and every SSN is unique, and every SSN appears in every file.

Make sure the SSN is not repeated 4 times - one time per row is enough!

##### Goal 3

Next, you want to identify any stale records, where stale simply means the record has not been updated since 3/1/2017 (e.g. last update date < 3/1/2017). Create an iterator that only contains current records (i.e. not stale) based on the `last_updated` field from the `status_update` file.

##### Goal 4

Find the largest group of car makes for each gender.

Possibly more than one such group per gender exists (equal sizes).

In [1]:
import csv
from itertools import chain, compress, groupby, tee

#### constants.py


#### Files

fname_employment = 'employment.csv'

fname_update_status = 'update_status.csv'

fname_vehicles = 'vehicles.csv'

fname_personal_info = 'personal_info.csv'


fnames = fname_employment, fname_update_status, fname_vehicles, fname_personal_info





#### Parsers

def parse_date(value, *, fmt='%Y-%m-%dT%H:%M:%SZ'):
    return datetime.strptime(value, fmt)

employment_parser = (str, str, str, str)

update_status_parser = (str, parse_date, parse_date)

vehicles_parser = (str, str, str, int)

personal_info_parser = (str, str, str, str, str)


parsers = employment_parser, update_status_parser, vehicles_parser, personal_info_parser
                


    
#### Named Tuple Names

employment_class_name = 'Employment'

update_status_class_name = 'UpdateStatus'

vehicles_class_name = 'Vehicles'

personal_class_name = 'Personal'


class_names = employment_class_name, update_status_class_name, vehicles_class_name, personal_class_name




#### Field Inclusion and Exclusion

employment_field_compress = [True, True, True, False]

update_status_fileld_compress = [False, True, True]

vehicle_field_compress = [False, True, True, True]

personal_field_compress = [True, True, True, True, True]


compress_fields = (employment_field_compress, update_status_fileld_compress, 
                   vehicle_field_compress, personal_field_compress
)

In [2]:
########## parse_utils.py

from datetime import datetime
from collections import namedtuple

def csv_parser(fname, *, delimiter=',', quotechar='"', include_reader=False):
    with open(fname) as f:
        reader = csv.reader(f, delimiter=delimiter, quotechar=quotechar)
        if not include_reader:
            next(f)
        yield from reader


def parse_date(value, *, fmt='%Y-%m-%dT%H:%M:%SZ'):
    return datetime.strptime(value, fmt)


def extract_field_names(fname):
    reader = csv_parser(fname, include_reader=True)
    return next(reader)


def create_namedtuple_class(fname, class_name):
    fields = extract_field_names(fname)
    return namedtuple(class_name, fields)

def create_combo_named_tuple_class(fnames, compress_fields):
    compress_fields = chain.from_iterable(compress_fields)
    
    field_names = chain.from_iterable(extract_field_names(fname) for fname in fnames)
    compressed_fields_names = compress(field_names, compress_fields)
    return namedtuple('Data', compressed_fields_names)

def iter_file(fname, class_name, parser):
    nt_class = create_namedtuple_class(fname, class_name)
    reader = csv_parser(fname)
    for row in reader:
        parsed_data = (parse_fn(value) for value, parse_fn in zip(row, parser))
        yield nt_class(*parsed_data)
        
        
def iter_combined_plain_tuple(fnames, class_names, parsers, compress_fields):
    
    compress_fields = list(chain.from_iterable(compress_fields))
    
    zipped_tuples = zip(*[iter_file(fname, class_name, parser)
               for fname, class_name, parser in zip(fnames, class_names, parsers)])
    
    merged_iter = (chain.from_iterable(zipped_tuple) for zipped_tuple in zipped_tuples)
    
    for row in merged_iter:
        compressed_row = compress(row, compress_fields)
        yield tuple(compressed_row)
        
        
        
        
def iter_combined(fnames, class_names, parsers, compress_fields):
    combo_nt = create_combo_named_tuple_class(fnames, compress_fields)
    compress_fields = list(chain.from_iterable(compress_fields))
    
    zipped_tuples = zip(*[iter_file(fname, class_name, parser)
               for fname, class_name, parser in zip(fnames, class_names, parsers)])
    
    merged_iter = (chain.from_iterable(zipped_tuple) for zipped_tuple in zipped_tuples)
    
    for row in merged_iter:
        compressed_row = compress(row, compress_fields)
        yield combo_nt(*compressed_row)

        
def filtered_iter_combined(fnames, class_names, parsers, compress_fields, *, key=None):
    iter_combo = iter_combined(fnames, class_names, parsers, compress_fields)
    
    yield from filter(key, iter_combo)
    
    
def group_data(fnames, class_names, parsers, compress_fields, filter_key, group_key, gender):
    data = filtered_iter_combined(fnames, class_names, parsers, compress_fields, key=filter_key)
    data_filtered = (row for row in data if row.gender == gender)
    sorted_data = sorted(data_filtered, key=group_key)
    groups = groupby(sorted_data, key=group_key)
    group_counts = ((g[0], len(list(g[1]))) for g in groups)
    return group_counts

In [3]:
data = filtered_iter_combined(fnames, class_names, parsers, compress_fields,
                                    key=lambda x: x.last_updated >= datetime(2017, 3, 1))

In [4]:
def group_key(item):
    return item.vehicle_make

In [5]:
# sorted_data = sorted(data, key=group_key)
# groups_1 = groupby(sorted_data, key=group_key)

# groups_2 = groupby(sorted_data, key=group_key)

In [6]:
# rg_1 = next(groups_1)
# rg_2 = next(groups_2)

# print(id(groups_1), rg_1, id(rg_1), rg_1[0], id(rg_1[0]), rg_1[1], id(rg_1[1]))
# print(id(groups_2), rg_2, id(rg_2), rg_2[0], id(rg_2[0]), rg_2[1], id(rg_2[1]))

In [7]:
###[(k, len(list(v))) for k, v in (groups)]

In [8]:
# group_f = (item for item in groups_1 if item[0][0] == 'Female')
# data_f = ((item[0][1], len(list(item[1]))) for item in group_f)

# print('group_f')
# for row in data_f:
#     print(row)

In [9]:
# group_m = (item for item in groups_2 if item[0][0] == 'Male')
# data_m = ((item[0][1], len(list(item[1]))) for item in group_m)

# print('group_m')
# for row in data_m:
#     print(row)

In [10]:
data_1, data_2 = tee(data, 2)

In [11]:
data_m = (row for row in data_1 if row.gender == 'Male')

sorted_data_m = sorted(data_m, key=group_key)
groups_m = groupby(sorted_data_m, key=group_key)
groups_m_count = ((g[0], len(list(g[1]))) for g in groups_m)
                   
print('groups_m')
for row in groups_m_count:
    print(row)
print()
print()



data_f = (row for row in data_2 if row.gender == 'Female')

sorted_data_f = sorted(data_f, key=group_key)
groups_f = groupby(sorted_data_f, key=group_key)
groups_f_count = ((g[0], len(list(g[1]))) for g in groups_f)

print('groups_f')
for row in groups_f_count:
    print(row)

groups_m
('Acura', 7)
('Aptera', 1)
('Aston Martin', 3)
('Audi', 14)
('Austin', 1)
('BMW', 12)
('Bentley', 3)
('Buick', 13)
('Cadillac', 9)
('Chevrolet', 30)
('Chrysler', 3)
('Corbin', 1)
('Daewoo', 1)
('Dodge', 22)
('Eagle', 1)
('Ford', 40)
('GMC', 28)
('Geo', 2)
('Honda', 9)
('Hyundai', 8)
('Infiniti', 7)
('Isuzu', 3)
('Jaguar', 4)
('Jeep', 7)
('Jensen', 1)
('Kia', 5)
('Lamborghini', 4)
('Land Rover', 3)
('Lexus', 6)
('Lincoln', 5)
('Lotus', 5)
('Maserati', 3)
('Maybach', 2)
('Mazda', 13)
('Mercedes-Benz', 19)
('Mercury', 11)
('Mitsubishi', 28)
('Nissan', 6)
('Oldsmobile', 5)
('Panoz', 2)
('Plymouth', 4)
('Pontiac', 11)
('Porsche', 4)
('Rolls-Royce', 1)
('Saab', 8)
('Saturn', 3)
('Scion', 1)
('Smart', 1)
('Subaru', 8)
('Suzuki', 2)
('Toyota', 21)
('Volkswagen', 16)
('Volvo', 10)


groups_f
('Acura', 9)
('Aston Martin', 2)
('Audi', 13)
('Austin', 1)
('BMW', 12)
('Bentley', 4)
('Bugatti', 1)
('Buick', 11)
('Cadillac', 6)
('Chevrolet', 42)
('Chrysler', 6)
('Dodge', 17)
('Eagle', 1)
('Fo

In [12]:
cutoff_date = datetime(2017, 3, 1)
result_m = group_data(fnames, class_names, parsers, compress_fields,
           filter_key=lambda row: row.last_updated >= cutoff_date,
          group_key=lambda row: row.vehicle_make,
          gender='Male')

In [13]:
for row in result_m:
    print(row)

('Acura', 7)
('Aptera', 1)
('Aston Martin', 3)
('Audi', 14)
('Austin', 1)
('BMW', 12)
('Bentley', 3)
('Buick', 13)
('Cadillac', 9)
('Chevrolet', 30)
('Chrysler', 3)
('Corbin', 1)
('Daewoo', 1)
('Dodge', 22)
('Eagle', 1)
('Ford', 40)
('GMC', 28)
('Geo', 2)
('Honda', 9)
('Hyundai', 8)
('Infiniti', 7)
('Isuzu', 3)
('Jaguar', 4)
('Jeep', 7)
('Jensen', 1)
('Kia', 5)
('Lamborghini', 4)
('Land Rover', 3)
('Lexus', 6)
('Lincoln', 5)
('Lotus', 5)
('Maserati', 3)
('Maybach', 2)
('Mazda', 13)
('Mercedes-Benz', 19)
('Mercury', 11)
('Mitsubishi', 28)
('Nissan', 6)
('Oldsmobile', 5)
('Panoz', 2)
('Plymouth', 4)
('Pontiac', 11)
('Porsche', 4)
('Rolls-Royce', 1)
('Saab', 8)
('Saturn', 3)
('Scion', 1)
('Smart', 1)
('Subaru', 8)
('Suzuki', 2)
('Toyota', 21)
('Volkswagen', 16)
('Volvo', 10)


In [14]:
gen = iter_combined(fnames, class_names, parsers, compress_fields)

In [15]:
for _ in range(10):
    print(next(gen))

Data(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30), vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993, ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Data(employer='Nicolas and Sons', department='Sales', employee_id='41-6841359', last_updated=datetime.datetime(2017, 1, 23, 11, 23, 17), created=datetime.datetime(2016, 1, 27, 4, 32, 57), vehicle_make='Ford', vehicle_model='Mustang', model_year=1997, ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
Data(employer='Connelly Group', department='Research and Development', employee_id='98-7952860', last_updated=datetime.datetime(2017, 10, 4, 11, 21, 30), created=datetime.datetime(2016, 9, 21, 23, 4, 7), vehicle_make='GMC', vehicle_model='Yukon', model_year=2005, ssn='101-84-